# Creation of advanced feature representation

In [11]:
import sys
import re
import os
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.models import Model
from tqdm import tqdm
import pyvips

In [2]:
all_data = pd.read_csv('/home/nelssalminen/painters/data/all_data_info_custom.csv')
train_data = pd.read_csv('/home/nelssalminen/painters/data/train_info.csv')
test_data = pd.read_csv('/home/nelssalminen/painters/data/test_info.csv')
IMG_DIR = '/home/nelssalminen/painters/fullres_data/'
OUTPUT_DIR ='/home/nelssalminen/painters/fullres_data/output/'

IMG_LIST = all_data["new_filename"].tolist()

TARGET_SIZE = 112
CORNER_ZOOM = 1.75

error_out = open(OUTPUT_DIR + 'error_images_4.tsv', "w")

In [ ]:
model = Xception(weights='imagenet',include_top=False,pooling='avg')

In [4]:
import pickle
with open ('outfile', 'rb') as fp:
    error_anyway = pickle.load(fp)

In [5]:
def gen_features(filename, image_segments, fvec):
    for s, segment in image_segments.items():
        x = np.expand_dims(segment, axis=0)
        x = preprocess_input(x)
        try:
            features = model.predict(x)[0]
        except ValueError:
            error_out.write(filename + '\n')
            features = np.array([-1])
            break
        #image_vector = ",".join(["{:.5e}".format(v) for v in features.tolist()])
        image_vector = ",".join(["{:.5e}".format(v) for v in features.tolist()])
        filepath, ext = filename.strip().split(".")
        fvec.write("{:s}\t{:s}\n".format(filepath + '_' + s + '.' + ext, image_vector))

In [6]:
def create_corner_crops(f, w=TARGET_SIZE, z=CORNER_ZOOM):
    try:
        i = pyvips.Image.new_from_file(f)
        a_roi = i.crop(0,0,w,w)
        a_mem = a_roi.write_to_memory()
        a = np.ndarray(buffer=a_mem, dtype=np.uint8,
                       shape=[a_roi.height, a_roi.width, a_roi.bands])
        b_roi = i.crop(i.width-w,0,w,w)
        b_mem = b_roi.write_to_memory()
        b = np.ndarray(buffer=b_mem, dtype=np.uint8,
                       shape=[b_roi.height, b_roi.width, b_roi.bands])
        c_roi = i.crop(0,i.height-w,w,w)
        c_mem = c_roi.write_to_memory()
        c = np.ndarray(buffer=c_mem, dtype=np.uint8,
                       shape=[c_roi.height, c_roi.width, c_roi.bands])
        d_roi = i.crop(i.width-w,i.height-w,w,w)
        d_mem = d_roi.write_to_memory()
        d = np.ndarray(buffer=d_mem, dtype=np.uint8,
                       shape=[d_roi.height, d_roi.width, d_roi.bands])
    except pyvips.error.Error:
        print(f + 'raised pyvips error')
        return np.array([]), np.array([]), np.array([]), np.array([])
    return a, b, c, d

In [8]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

IMG_LIST_SPLIT = list(split(IMG_LIST, 2))

In [ ]:
fvec = open(OUTPUT_DIR + 'xception-segment-vectors-gpu-4.tsv', "w")
for f in tqdm(error_anyway):
    a, b, c, d = create_corner_crops(IMG_DIR + f)
    if a.size != 0:
        gen_features(f, {'a': a, 'b': b, 'c': c, 'd': d}, fvec)
    else:
        error_out.write(filename + '\n')